In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import json
import time
import requests, sys

In [5]:
os.chdir('..')

In [35]:
import pandas as pd
from rnamotif.connectors.get_ensembl import get_ensembl_seq_by_gene
from rnamotif.utility.ensembl_rna_handler import save_one_fasta_from_ensembl_api_return

In [6]:
import warnings
warnings.filterwarnings('ignore')

#### Download sequence from Ensembl

In [7]:
df_s1 = pd.read_excel(os.path.join(os.path.abspath(r'./samples'),r'aan2399_table_S1.xlsx'))

In [15]:
lst_gene = list(df_s1['target_id'])

In [ ]:
res = get_ensembl_seq_by_gene(lst_gene,seq_type='cds',is_ssl_verification=False)

In [19]:
save_one_fasta_from_ensembl_api_return(os.path.abspath(r'./samples'), 's1_cds.fasta', res1)

FASTA file: D:\workspace\rnamotif\samples\s1_cds_1.fasta


True

#### Load Fasta file

In [11]:
cdna_file = os.path.join(os.path.abspath(r'./samples'), 's1_cdna.fasta')
cds_file = os.path.join(os.path.abspath(r'./samples'), 's1_cds.fasta')
cdna_zipfile = os.path.join(os.path.abspath(r'./samples'), 's1_cdna.zip')
cds_zipfile = os.path.join(os.path.abspath(r'./samples'), 's1_cds.zip')

In [29]:
with open(cdna_file, 'r') as fin:
    cdna_data = fin.read().splitlines()

with open(cds_file, 'r') as fin:
    cds_data = fin.read().splitlines()

In [8]:
from zipfile import ZipFile

In [16]:
with ZipFile(cdna_zipfile, 'r') as zip_ref:
    zip_ref.extract(os.path.basename(cdna_file),os.path.dirname(cdna_file))

with ZipFile(cds_zipfile, 'r') as zip_ref:
    zip_ref.extract(os.path.basename(cds_file),os.path.dirname(cds_file))

In [17]:
from Bio import SeqIO

In [26]:
dic_cds = dict()
with open(cds_file, "r") as fin:
    for rec in SeqIO.parse(fin, "fasta"):
        dic_cds[rec.id] =rec

In [27]:
dic_cdna = dict()
with open(cdna_file, "r") as fin:
    for rec in SeqIO.parse(fin, "fasta"):
        dic_cdna[rec.id] =rec

#### Extract 3'UTR 

In [68]:
from Bio.Seq import Seq

In [76]:
mismatch = list()
errfind = list()
dic_utr3 = dict()
for cdskey in dic_cds:
    _cds = dic_cds.get(cdskey)
    _cdna = dic_cdna.get(cdskey)
    start_ix = _cdna.seq.find(_cds.seq)
    if start_ix < 0:
        mismatch.append(cdskey)
    else:
        _utr3 = _cdna.seq[start_ix + len(_cds.seq):]
        # It should be Zero
        length_verified = len(_cdna.seq) - start_ix - len(_cds.seq) - len(_utr3)
        if length_verified != 0:
            errfind.append((cdskey, length_verified))
        else:
            dic_utr3[cdskey] = _utr3
print(f'# CDNA: {len(dic_cdna)}, # CDS: {len(dic_cds)}, # mismatch: {len(mismatch)}, # errfind: {len(errfind)}, # UTR3: {len(dic_utr3)}')

# CDNA: 9858, # CDS: 7784, # mismatch: 235, # errfind: 0, # UTR3: 7549


#### Example: Mismatch

In [54]:
print('https://rest.ensembl.org/sequence/id/ENSMUST00000178282?type=cds')

https://rest.ensembl.org/sequence/id/ENSMUST00000178282?type=cds


In [55]:
print('https://rest.ensembl.org/sequence/id/ENSMUST00000178282?type=cdna')

https://rest.ensembl.org/sequence/id/ENSMUST00000178282?type=cdna
